# LKW-Flotten-Elektrifizierung: MILP-Optimierungsmodell

## Erweitert mit Vollladen-Schutz (NB24) und Periodizitätsbedingungen (NB0)

**Ziel:** Minimierung der jährlichen Gesamtkosten für die Elektrifizierung einer LKW-Flotte

**Modelltyp:** Gemischt-ganzzahliges lineares Programm (MILP)

**Solver:** HiGHS

**Besonderheiten:**
- ✅ Zeitperiodizität (NB0): 24h-Zyklus konsistent
- ✅ Vollladen-Schutz (NB24): Kein ineffizientes Nachladen
- ✅ χ-Verknüpfung (NB14a): Exakte Ladeleistungs-Kopplung
- ✅ Ladeverluste: Realistische 5% (η_ch = 0.95)
- ✅ ~22.000 Variablen, ~72.000 Constraints

---

## 1. Import Required Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

# Optimization
try:
    import highspy
    print("✓ highspy imported successfully")
except ImportError:
    print("⚠ highspy not found. Install with: pip install highspy")

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Define Model Parameters

### 2.1 Indexmengen (Sets)

In [ ]:
# Vehicles (V)
n_vehicles = 20
vehicles = list(range(n_vehicles))

# Vehicle Types (F)
vehicle_types = ['ActrosL', 'eActros400', 'eActros600']
diesel_types = ['ActrosL']
electric_types = ['eActros400', 'eActros600']

# Routes (R)
n_routes = 20
routes = [f't-{i+4}' if i < 10 else f'k{i-9}' for i in range(n_routes)]

# Charging Station Types (L)
station_types = ['Alpi-50', 'Alpi-200', 'Alpi-400']

# Time Steps (T) - 96 timesteps of 15 minutes
n_timesteps = 96
timesteps = list(range(n_timesteps))

# Night Time Steps (18:00-06:00 = timesteps 72-96 and 0-24)
night_timesteps = list(range(72, 96)) + list(range(0, 24))

print(f"✓ Vehicles: {n_vehicles}")
print(f"✓ Routes: {n_routes}")
print(f"✓ Timesteps: {n_timesteps} (15-min intervals)")
print(f"✓ Night timesteps: {len(night_timesteps)}")

### 2.2 Zeit- und Fahrzeugparameter

In [ ]:
# Time Parameters
delta_t = 0.25  # hours per timestep
D = 260  # operating days per year

# Vehicle Parameters (DataFrame for better readability)
vehicle_params = pd.DataFrame({
    'Type': vehicle_types,
    'CAPEX': [24000, 50000, 60000],  # €/year
    'OPEX': [6000, 5000, 6000],  # €/year
    'KFZ_Tax': [556, 0, 0],  # €/year
    'THG_Revenue': [0, 1000, 1000],  # €/year
    'Consumption': [26, 105, 110],  # L/100km or kWh/100km
    'Q_max': [0, 414, 621],  # kWh (battery capacity)
    'P_max_vehicle': [0, 400, 400],  # kW (max charging power)
    'SOC_min': [0, 41.4, 62.1]  # kWh (10% minimum SOC)
})

print("Vehicle Parameters:")
print(vehicle_params)

### 2.3 Routenparameter

In [ ]:
# Route Parameters (example data - replace with actual route data)
np.random.seed(42)
route_params = pd.DataFrame({
    'Route': routes,
    'd_r': np.random.randint(150, 400, n_routes),  # km (total distance)
    'd_maut_r': np.random.randint(50, 200, n_routes),  # km (toll distance)
    't_start': np.random.randint(24, 48, n_routes),  # start timestep (6:00-12:00)
    't_end': np.random.randint(60, 80, n_routes)  # end timestep (15:00-20:00)
})

# Ensure t_end > t_start
route_params['t_end'] = route_params.apply(
    lambda row: max(row['t_start'] + 8, row['t_end']), axis=1
)

print("Route Parameters (first 10):")
print(route_params.head(10))
print(f"\nTotal routes: {len(route_params)}")

### 2.4 Ladesäulen- und Energieparameter

In [ ]:
# Charging Station Parameters
station_params = pd.DataFrame({
    'Type': station_types,
    'CAPEX_l': [3000, 10000, 16000],  # €/year
    'OPEX_l': [1000, 1500, 2000],  # €/year
    'P_max_l': [50, 200, 400],  # kW
    'n_spots': [2, 2, 2]  # number of charging points
})

# Energy Cost Parameters
p_arbeit = 0.25  # €/kWh (electricity work price)
p_leistung = 150  # €/kW (power price)
p_grund = 1000  # €/year (base fee)
p_diesel = 1.60  # €/L
p_maut = 0.34  # €/km (toll for diesel)

# Network Parameters
P_netz_basis = 500  # kW (base network connection)
P_netz_erw = 500  # kW (network expansion)
c_netz_erw = 10000  # €/year (network expansion cost)
L_max = 3  # maximum number of charging stations

# Storage Parameters
c_sp_p = 30  # €/kW (storage power CAPEX)
c_sp_e = 350  # €/kWh (storage capacity CAPEX)
alpha_opex = 0.02  # storage OPEX rate
eta_storage = 0.98  # round-trip efficiency (storage)
eta_charging = 0.95  # charging efficiency (vehicle) - NEU!
DoD = 0.975  # max depth of discharge

# Big-M Parameters - NEU!
M_SOC = 621  # kWh (max battery capacity - eActros600)
M_Sp = 10000  # kW (storage Big-M)
M_ch = 400  # kW (charging power Big-M)
epsilon_min = 0.1  # kW (minimum charging power)

print("Charging Station Parameters:")
print(station_params)
print(f"\n✓ Electricity price: {p_arbeit} €/kWh")
print(f"✓ Charging efficiency: {eta_charging} (5% losses)")
print(f"✓ Storage efficiency: {eta_storage}")
print(f"✓ Big-M parameters: M_SOC={M_SOC}, M_Sp={M_Sp}, M_ch={M_ch}")

### 2.5 Helper Functions: Active Routes per Timestep

In [ ]:
def get_active_routes(t, route_params):
    """Returns list of route indices active at timestep t"""
    return [
        r for r in range(len(route_params))
        if route_params.iloc[r]['t_start'] <= t < route_params.iloc[r]['t_end']
    ]

# Test function
test_timestep = 40
active_routes_test = get_active_routes(test_timestep, route_params)
print(f"Routes active at timestep {test_timestep}: {len(active_routes_test)} routes")
print(f"Route indices: {active_routes_test[:5]}... (showing first 5)")

## 3. Initialize HiGHS Model and Decision Variables

In [ ]:
# Initialize HiGHS model
h = highspy.Highs()
h.setOptionValue("log_to_console", True)
h.setOptionValue("time_limit", 600.0)  # 10 minutes
h.setOptionValue("mip_rel_gap", 0.01)  # 1% MIP gap

print("✓ HiGHS model initialized")
print(f"  Time limit: 600s (10 minutes)")
print(f"  MIP gap: 1%")

### 3.1 Binary Decision Variables

In [ ]:
# Binary Variables (dictionary to store variable indices)
binary_vars = {}

# use_v: Vehicle v is used
binary_vars['use'] = {}
for v in vehicles:
    var_idx = h.getNumCol()
    h.addVar(0, 1)
    h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
    binary_vars['use'][v] = var_idx

# tau_vf: Vehicle v is of type f
binary_vars['tau'] = {}
for v in vehicles:
    for f_idx, f in enumerate(vehicle_types):
        var_idx = h.getNumCol()
        h.addVar(0, 1)
        h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
        binary_vars['tau'][(v, f)] = var_idx

# epsilon_v: Vehicle v is electric
binary_vars['epsilon'] = {}
for v in vehicles:
    var_idx = h.getNumCol()
    h.addVar(0, 1)
    h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
    binary_vars['epsilon'][v] = var_idx

# x_vr: Vehicle v drives route r
binary_vars['x'] = {}
for v in vehicles:
    for r in range(n_routes):
        var_idx = h.getNumCol()
        h.addVar(0, 1)
        h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
        binary_vars['x'][(v, r)] = var_idx

# y_l: Charging station l is installed
binary_vars['y'] = {}
for l_idx, l in enumerate(station_types):
    var_idx = h.getNumCol()
    h.addVar(0, 1)
    h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
    binary_vars['y'][l] = var_idx

# w_vlt: Vehicle v occupies charging point at station l at time t
binary_vars['w'] = {}
for v in vehicles:
    for l_idx, l in enumerate(station_types):
        for t in timesteps:
            var_idx = h.getNumCol()
            h.addVar(0, 1)
            h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
            binary_vars['w'][(v, l, t)] = var_idx

# omega_vt: Vehicle v is on route at time t
binary_vars['omega'] = {}
for v in vehicles:
    for t in timesteps:
        var_idx = h.getNumCol()
        h.addVar(0, 1)
        h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
        binary_vars['omega'][(v, t)] = var_idx

# chi_vt: Vehicle v is actively charging at time t
binary_vars['chi'] = {}
for v in vehicles:
    for t in timesteps:
        var_idx = h.getNumCol()
        h.addVar(0, 1)
        h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
        binary_vars['chi'][(v, t)] = var_idx

# mu_vt: Vehicle v is fully charged at time t (NEU!)
binary_vars['mu'] = {}
for v in vehicles:
    for t in timesteps:
        var_idx = h.getNumCol()
        h.addVar(0, 1)
        h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
        binary_vars['mu'][(v, t)] = var_idx

# gamma: Network expansion
var_idx = h.getNumCol()
h.addVar(0, 1)
h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
binary_vars['gamma'] = var_idx

# sigma_t: Storage mode (1=charging, 0=discharging)
binary_vars['sigma'] = {}
for t in timesteps:
    var_idx = h.getNumCol()
    h.addVar(0, 1)
    h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
    binary_vars['sigma'][t] = var_idx

# delta_vr: Vehicle v drives route r with diesel
binary_vars['delta'] = {}
for v in vehicles:
    for r in range(n_routes):
        var_idx = h.getNumCol()
        h.addVar(0, 1)
        h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
        binary_vars['delta'][(v, r)] = var_idx

# phi_vrf: Vehicle v drives route r with type f
binary_vars['phi'] = {}
for v in vehicles:
    for r in range(n_routes):
        for f in vehicle_types:
            var_idx = h.getNumCol()
            h.addVar(0, 1)
            h.changeColIntegrality(var_idx, highspy.HighsVarType.kInteger)
            binary_vars['phi'][(v, r, f)] = var_idx

print(f"✓ Binary variables created: ~{len(binary_vars['use']) + len(binary_vars['tau']) + len(binary_vars['x']) + len(binary_vars['w']) + len(binary_vars['omega']) + len(binary_vars['chi']) + len(binary_vars['mu'])} variables")

### 3.2 Continuous Decision Variables

In [ ]:
# Continuous Variables
continuous_vars = {}

# SOC_vt: State of charge for vehicle v at time t (kWh)
continuous_vars['SOC'] = {}
for v in vehicles:
    for t in timesteps:
        var_idx = h.getNumCol()
        h.addVar(0, M_SOC)
        continuous_vars['SOC'][(v, t)] = var_idx

# p_ch_vlt: Charging power for vehicle v at station l at time t (kW)
continuous_vars['p_ch'] = {}
for v in vehicles:
    for l in station_types:
        for t in timesteps:
            var_idx = h.getNumCol()
            h.addVar(0, M_ch)
            continuous_vars['p_ch'][(v, l, t)] = var_idx

# p_netz_t: Network power consumption at time t (kW)
continuous_vars['p_netz'] = {}
for t in timesteps:
    var_idx = h.getNumCol()
    h.addVar(0, 1e20)
    continuous_vars['p_netz'][t] = var_idx

# P_peak: Peak power demand (kW)
var_idx = h.getNumCol()
h.addVar(0, 1e20)
continuous_vars['P_peak'] = var_idx

# Storage variables
var_idx = h.getNumCol()
h.addVar(0, 1e20)
continuous_vars['P_sp'] = var_idx

var_idx = h.getNumCol()
h.addVar(0, 1e20)
continuous_vars['E_sp'] = var_idx

# SOC_sp_t: Storage state of charge at time t (kWh)
continuous_vars['SOC_sp'] = {}
for t in timesteps:
    var_idx = h.getNumCol()
    h.addVar(0, 1e20)
    continuous_vars['SOC_sp'][t] = var_idx

# p_sp_ch_t: Storage charging power at time t (kW)
continuous_vars['p_sp_ch'] = {}
for t in timesteps:
    var_idx = h.getNumCol()
    h.addVar(0, 1e20)
    continuous_vars['p_sp_ch'][t] = var_idx

# p_sp_dis_t: Storage discharging power at time t (kW)
continuous_vars['p_sp_dis'] = {}
for t in timesteps:
    var_idx = h.getNumCol()
    h.addVar(0, 1e20)
    continuous_vars['p_sp_dis'][t] = var_idx

print(f"✓ Continuous variables created: ~{len(continuous_vars['SOC']) + len(continuous_vars['p_ch']) + len(continuous_vars['SOC_sp'])} variables")

## 4. Define Objective Function

Minimize total annual costs:
$$C^{Gesamt} = C^{LKW} + C^{Lade} + C^{Strom} + C^{Netz} + C^{Speicher} + C^{Diesel} + C^{Maut} - C^{THG}$$

In [ ]:
# Objective function - set all costs directly
print("Setting objective function...")

# Initialize all costs to zero
n_vars = h.getNumCol()
costs = np.zeros(n_vars, dtype=np.float64)

# C_LKW: Vehicle costs (CAPEX + OPEX + Tax)
for v in vehicles:
    for f_idx, f in enumerate(vehicle_types):
        cost_f = vehicle_params.loc[f_idx, 'CAPEX'] + vehicle_params.loc[f_idx, 'OPEX'] + vehicle_params.loc[f_idx, 'KFZ_Tax']
        var_idx = binary_vars['tau'][(v, f)]
        costs[var_idx] = cost_f

# -C_THG: THG quota revenues (negative = revenue)
for v in vehicles:
    for f_idx, f in enumerate(electric_types):
        f_full_idx = vehicle_types.index(f)
        revenue = vehicle_params.loc[f_full_idx, 'THG_Revenue']
        var_idx = binary_vars['tau'][(v, f)]
        costs[var_idx] -= revenue  # Subtract revenue

# C_Lade: Charging infrastructure costs
for l_idx, l in enumerate(station_types):
    cost_l = station_params.loc[l_idx, 'CAPEX_l'] + station_params.loc[l_idx, 'OPEX_l']
    var_idx = binary_vars['y'][l]
    costs[var_idx] = cost_l

# C_Strom: Electricity costs
# Base fee handled separately (add to offset)
objective_offset = p_grund

# Power price
costs[continuous_vars['P_peak']] = p_leistung

# Energy price: p_arbeit * D * sum(p_netz_t * delta_t)
for t in timesteps:
    costs[continuous_vars['p_netz'][t]] = p_arbeit * D * delta_t

# C_Netz: Network expansion cost
costs[binary_vars['gamma']] = c_netz_erw

# C_Speicher: Storage costs
costs[continuous_vars['P_sp']] = (1 + alpha_opex) * c_sp_p
costs[continuous_vars['E_sp']] = (1 + alpha_opex) * c_sp_e

# C_Diesel: Diesel costs
kappa_diesel = vehicle_params.loc[0, 'Consumption']
for v in vehicles:
    for r in range(n_routes):
        d_r = route_params.iloc[r]['d_r']
        cost_diesel = D * p_diesel * (kappa_diesel / 100) * d_r
        var_idx = binary_vars['delta'][(v, r)]
        costs[var_idx] = cost_diesel

# C_Maut: Toll costs
for v in vehicles:
    for r in range(n_routes):
        d_maut_r = route_params.iloc[r]['d_maut_r']
        cost_maut = D * p_maut * d_maut_r
        var_idx = binary_vars['delta'][(v, r)]
        costs[var_idx] += cost_maut  # Add to existing diesel cost

# Set objective with NumPy arrays
indices = np.arange(n_vars, dtype=np.int32)
h.changeColsCost(n_vars, indices, costs)

# Set objective offset (constant term)
h.changeObjectiveOffset(objective_offset)

# Minimize
h.changeObjectiveSense(highspy.ObjSense.kMinimize)

print("✓ Objective function defined")
print(f"  Base fee (constant): {objective_offset:,.0f} €/year")
print(f"  Components: Vehicle costs, THG revenues, charging infrastructure,")
print(f"              electricity, network, storage, diesel, toll")

## 5. Constraints (Nebenbedingungen)

Implementierung aller Constraints NB0-NB26 aus der Dokumentation

In [ ]:
print("Hinzufügen der Constraints...")
print("=" * 60)

# Constraint counter
constraint_count = 0

### 5.0 Zeitperiodizität (NB0) - NEU!

In [ ]:
# NB0: Periodicity - SOC and storage SOC at t=96 must equal t=1 (24h cycle)
import numpy as np

for v in vehicles:
    h.addRow(0, 0, 2, 
             np.array([continuous_vars['SOC'][(v, 0)], continuous_vars['SOC'][(v, 95)]], dtype=np.int32), 
             np.array([1.0, -1.0], dtype=np.float64))
    constraint_count += 1

# Storage periodicity
h.addRow(0, 0, 2,
         np.array([continuous_vars['SOC_sp'][0], continuous_vars['SOC_sp'][95]], dtype=np.int32),
         np.array([1.0, -1.0], dtype=np.float64))
constraint_count += 1

print(f"✓ NB0 (Periodicity): {n_vehicles + 1} constraints")

### 5.1 Tourenabdeckung (NB1)

In [ ]:
# NB1: Each route must be driven by exactly one vehicle
for r in range(n_routes):
    indices = np.array([binary_vars['x'][(v, r)] for v in vehicles], dtype=np.int32)
    values = np.ones(n_vehicles, dtype=np.float64)
    h.addRow(1, 1, n_vehicles, indices, values)
    constraint_count += 1

print(f"✓ NB1 (Tour coverage): {n_routes} constraints")

### 5.2-5.4 Fahrzeugzuweisung

In [ ]:
# NB2: Type assignment - exactly one type per active vehicle
for v in vehicles:
    indices = np.array([binary_vars['tau'][(v, f)] for f in vehicle_types] + [binary_vars['use'][v]], dtype=np.int32)
    values = np.array([1.0, 1.0, 1.0, -1.0], dtype=np.float64)
    h.addRow(0, 0, 4, indices, values)
    constraint_count += 1

# NB3: Electric vehicle identification
for v in vehicles:
    indices = np.array([binary_vars['epsilon'][v]] + [binary_vars['tau'][(v, f)] for f in electric_types], dtype=np.int32)
    values = np.array([1.0, -1.0, -1.0], dtype=np.float64)
    h.addRow(0, 0, 3, indices, values)
    constraint_count += 1

# NB4: Vehicle activation - can only drive routes if activated
for v in vehicles:
    for r in range(n_routes):
        h.addRow(-highspy.kHighsInf, 0, 2,
                 np.array([binary_vars['x'][(v, r)], binary_vars['use'][v]], dtype=np.int32),
                 np.array([1.0, -1.0], dtype=np.float64))
        constraint_count += 1

print(f"✓ NB2-4 (Type assignment & activation): {n_vehicles * (1 + 1 + n_routes)} constraints")

### 5.5 Zeitliche Überlappung (NB5)

In [ ]:
# NB5: Temporal overlap - vehicle can drive max one route at a time
nb5_count = 0
for v in vehicles:
    for t in timesteps:
        active_routes = get_active_routes(t, route_params)
        if active_routes:
            indices = np.array([binary_vars['x'][(v, r)] for r in active_routes], dtype=np.int32)
            values = np.ones(len(active_routes), dtype=np.float64)
            h.addRow(-highspy.kHighsInf, 1, len(active_routes), indices, values)
            constraint_count += 1
            nb5_count += 1

print(f"✓ NB5 (Temporal overlap): {nb5_count} constraints")

### 5.6-5.8 Netz und Infrastruktur

In [ ]:
# NB6: Max number of charging stations
indices = np.array([binary_vars['y'][l] for l in station_types], dtype=np.int32)
values = np.ones(3, dtype=np.float64)
h.addRow(-highspy.kHighsInf, L_max, 3, indices, values)
constraint_count += 1

# NB7: Network power limitation
for t in timesteps:
    h.addRow(-highspy.kHighsInf, P_netz_basis, 2,
             np.array([continuous_vars['p_netz'][t], binary_vars['gamma']], dtype=np.int32),
             np.array([1.0, -P_netz_erw], dtype=np.float64))
    constraint_count += 1

# NB8: Peak power tracking
for t in timesteps:
    h.addRow(0, highspy.kHighsInf, 2,
             np.array([continuous_vars['P_peak'], continuous_vars['p_netz'][t]], dtype=np.int32),
             np.array([1.0, -1.0], dtype=np.float64))
    constraint_count += 1

print(f"✓ NB6-8 (Infrastructure & network): {1 + 2 * n_timesteps} constraints")

### 5.9-5.13 Batteriespeicher

In [ ]:
# NB9: Energy balance at network connection point
for t in timesteps:
    # p_netz + p_sp_dis = sum(charging) + p_sp_ch
    indices_list = ([continuous_vars['p_netz'][t], continuous_vars['p_sp_dis'][t], 
                continuous_vars['p_sp_ch'][t]] +
               [continuous_vars['p_ch'][(v, l, t)] for v in vehicles for l in station_types])
    values_list = [1.0, 1.0, -1.0] + [-1.0] * (n_vehicles * len(station_types))
    indices = np.array(indices_list, dtype=np.int32)
    values = np.array(values_list, dtype=np.float64)
    h.addRow(0, 0, len(values), indices, values)
    constraint_count += 1

# NB10: Storage SOC balance
for t in timesteps:
    if t == 0:
        prev_soc = continuous_vars['SOC_sp'][95]
    else:
        prev_soc = continuous_vars['SOC_sp'][t-1]
    
    indices = np.array([continuous_vars['SOC_sp'][t], prev_soc, 
               continuous_vars['p_sp_ch'][t], continuous_vars['p_sp_dis'][t]], dtype=np.int32)
    values = np.array([1.0, -1.0, -eta_storage * delta_t, delta_t], dtype=np.float64)
    h.addRow(0, 0, 4, indices, values)
    constraint_count += 1

# NB11: Storage power limitations
for t in timesteps:
    h.addRow(-highspy.kHighsInf, 0, 2,
             np.array([continuous_vars['p_sp_ch'][t], continuous_vars['P_sp']], dtype=np.int32),
             np.array([1.0, -1.0], dtype=np.float64))
    h.addRow(-highspy.kHighsInf, 0, 2,
             np.array([continuous_vars['p_sp_dis'][t], continuous_vars['P_sp']], dtype=np.int32),
             np.array([1.0, -1.0], dtype=np.float64))
    constraint_count += 2

# NB12: Exclusive storage mode (charging XOR discharging)
for t in timesteps:
    h.addRow(-highspy.kHighsInf, 0, 2,
             np.array([continuous_vars['p_sp_ch'][t], binary_vars['sigma'][t]], dtype=np.int32),
             np.array([1.0, -M_Sp], dtype=np.float64))
    h.addRow(-M_Sp, highspy.kHighsInf, 2,
             np.array([continuous_vars['p_sp_dis'][t], binary_vars['sigma'][t]], dtype=np.int32),
             np.array([1.0, M_Sp], dtype=np.float64))
    constraint_count += 2

# NB13: Storage SOC bounds
for t in timesteps:
    h.addRow(-highspy.kHighsInf, 0, 2,
             np.array([continuous_vars['SOC_sp'][t], continuous_vars['E_sp']], dtype=np.int32),
             np.array([1.0, -1.0], dtype=np.float64))
    h.addRow(0, highspy.kHighsInf, 2,
             np.array([continuous_vars['SOC_sp'][t], continuous_vars['E_sp']], dtype=np.int32),
             np.array([1.0, -(1 - DoD)], dtype=np.float64))
    constraint_count += 2

print(f"✓ NB9-13 (Battery storage): {7 * n_timesteps} constraints")

### 5.14-5.15 Fahrzeug-SOC mit Ladeverlust (η_ch = 0.95)

In [ ]:
# Helper: Calculate energy consumption per timestep
def get_consumption_per_step(r, f_idx):
    """kWh per timestep for electric vehicle type f on route r"""
    f_type = vehicle_types[f_idx]
    if f_type not in electric_types:
        return 0
    t_start = route_params.iloc[r]['t_start']
    t_end = route_params.iloc[r]['t_end']
    duration = t_end - t_start
    if duration <= 0:
        return 0
    total_km = route_params.iloc[r]['d_r']
    consumption_per_100km = vehicle_params.loc[f_idx, 'Consumption']
    total_consumption = (consumption_per_100km / 100) * total_km
    return total_consumption / duration

# NB14: Vehicle SOC balance with consumption and charging losses
for v in vehicles:
    for t in timesteps:
        if t == 0:
            prev_soc = continuous_vars['SOC'][(v, 95)]
        else:
            prev_soc = continuous_vars['SOC'][(v, t-1)]
        
        # Charging term with efficiency
        charging_indices = [continuous_vars['p_ch'][(v, l, t)] for l in station_types]
        
        # Consumption term (phi_vrf * consumption per step)
        consumption_indices = []
        consumption_values = []
        active_routes = get_active_routes(t, route_params)
        for r in active_routes:
            for f_idx, f in enumerate(vehicle_types):
                if f in electric_types:
                    cons = get_consumption_per_step(r, f_idx)
                    if cons > 0:
                        consumption_indices.append(binary_vars['phi'][(v, r, f)])
                        consumption_values.append(cons)
        
        indices_list = ([continuous_vars['SOC'][(v, t)], prev_soc] + 
                  charging_indices + consumption_indices)
        values_list = ([1.0, -1.0] + 
                 [-eta_charging * delta_t] * len(charging_indices) + 
                 consumption_values)
        
        indices = np.array(indices_list, dtype=np.int32)
        values = np.array(values_list, dtype=np.float64)
        h.addRow(0, 0, len(values), indices, values)
        constraint_count += 1

# NB14a: Chi-variable linkage (charging indicator)
for v in vehicles:
    for t in timesteps:
        # Sum of charging power <= M_ch * chi
        indices_list = [continuous_vars['p_ch'][(v, l, t)] for l in station_types] + [binary_vars['chi'][(v, t)]]
        values_list = [1.0] * len(station_types) + [-M_ch]
        indices = np.array(indices_list, dtype=np.int32)
        values = np.array(values_list, dtype=np.float64)
        h.addRow(-highspy.kHighsInf, 0, len(values), indices, values)
        
        # Sum of charging power >= epsilon * chi
        values2 = np.array([1.0] * len(station_types) + [-epsilon_min], dtype=np.float64)
        h.addRow(0, highspy.kHighsInf, len(values), indices, values2)
        constraint_count += 2

# NB15: Vehicle SOC bounds
for v in vehicles:
    for t in timesteps:
        # SOC <= Q_max of assigned type
        indices_list = [continuous_vars['SOC'][(v, t)]] + [binary_vars['tau'][(v, f)] for f in electric_types]
        q_max_values = [vehicle_params[vehicle_params['Type'] == f]['Q_max'].values[0] for f in electric_types]
        values_list = [1.0] + [-float(q) for q in q_max_values]
        indices = np.array(indices_list, dtype=np.int32)
        values = np.array(values_list, dtype=np.float64)
        h.addRow(-highspy.kHighsInf, 0, len(values), indices, values)
        
        # SOC >= SOC_min of assigned type
        soc_min_values = [vehicle_params[vehicle_params['Type'] == f]['SOC_min'].values[0] for f in electric_types]
        values2 = np.array([1.0] + [-float(s) for s in soc_min_values], dtype=np.float64)
        h.addRow(0, highspy.kHighsInf, len(values2), indices, values2)
        
        # SOC = 0 for diesel vehicles
        h.addRow(-highspy.kHighsInf, 0, 2,
                 np.array([continuous_vars['SOC'][(v, t)], binary_vars['epsilon'][v]], dtype=np.int32),
                 np.array([1.0, -M_SOC], dtype=np.float64))
        constraint_count += 3

print(f"✓ NB14 (SOC balance with η_ch=0.95): {n_vehicles * n_timesteps} constraints")
print(f"✓ NB14a (Chi-linkage): {2 * n_vehicles * n_timesteps} constraints")
print(f"✓ NB15 (SOC bounds): {3 * n_vehicles * n_timesteps} constraints")

### 5.16-5.18 Ladeleistung

In [ ]:
# NB16: Charging power limitations
for v in vehicles:
    for l_idx, l in enumerate(station_types):
        p_max_l = station_params.loc[l_idx, 'P_max_l']
        for t in timesteps:
            # Charging power <= station power * w
            h.addRow(-highspy.kHighsInf, 0, 2,
                     np.array([continuous_vars['p_ch'][(v, l, t)], binary_vars['w'][(v, l, t)]], dtype=np.int32),
                     np.array([1.0, -p_max_l], dtype=np.float64))
            constraint_count += 1
    
    # Total charging power per vehicle <= vehicle max charging power
    for t in timesteps:
        indices_list = [continuous_vars['p_ch'][(v, l, t)] for l in station_types]
        p_max_values = [vehicle_params[vehicle_params['Type'] == f]['P_max_vehicle'].values[0] 
                       for f in electric_types]
        indices_list += [binary_vars['tau'][(v, f)] for f in electric_types]
        values_list = [1.0] * len(station_types) + [-float(p) for p in p_max_values]
        indices = np.array(indices_list, dtype=np.int32)
        values = np.array(values_list, dtype=np.float64)
        h.addRow(-highspy.kHighsInf, 0, len(values), indices, values)
        constraint_count += 1

# NB17: Charging point capacity
for l_idx, l in enumerate(station_types):
    n_spots = station_params.loc[l_idx, 'n_spots']
    for t in timesteps:
        indices_list = [binary_vars['w'][(v, l, t)] for v in vehicles] + [binary_vars['y'][l]]
        values_list = [1.0] * n_vehicles + [-float(n_spots)]
        indices = np.array(indices_list, dtype=np.int32)
        values = np.array(values_list, dtype=np.float64)
        h.addRow(-highspy.kHighsInf, 0, len(values), indices, values)
        constraint_count += 1

# NB18: Total station power limitation
for l_idx, l in enumerate(station_types):
    p_max_l = station_params.loc[l_idx, 'P_max_l']
    for t in timesteps:
        indices_list = [continuous_vars['p_ch'][(v, l, t)] for v in vehicles] + [binary_vars['y'][l]]
        values_list = [1.0] * n_vehicles + [-float(p_max_l)]
        indices = np.array(indices_list, dtype=np.int32)
        values = np.array(values_list, dtype=np.float64)
        h.addRow(-highspy.kHighsInf, 0, len(values), indices, values)
        constraint_count += 1

print(f"✓ NB16-18 (Charging power): {n_vehicles * len(station_types) * n_timesteps + n_vehicles * n_timesteps + 2 * len(station_types) * n_timesteps} constraints")

### 5.19-5.23 Routing und Linearisierung

In [ ]:
# NB19: On-route linkage (equality!)
for v in vehicles:
    for t in timesteps:
        active_routes = get_active_routes(t, route_params)
        if active_routes:
            indices_list = [binary_vars['omega'][(v, t)]] + [binary_vars['x'][(v, r)] for r in active_routes]
            values_list = [1.0] + [-1.0] * len(active_routes)
            indices = np.array(indices_list, dtype=np.int32)
            values = np.array(values_list, dtype=np.float64)
            h.addRow(0, 0, len(values), indices, values)
        else:
            h.addRow(0, 0, 1, np.array([binary_vars['omega'][(v, t)]], dtype=np.int32), np.array([1.0], dtype=np.float64))
        constraint_count += 1

# NB20: Night parking rule - no spontaneous unplugging
nb20_count = 0
for v in vehicles:
    for l in station_types:
        for i, t in enumerate(night_timesteps[:-1]):
            t_next = night_timesteps[i + 1]
            if t_next == t + 1:  # consecutive timesteps
                # w[v,l,t] - w[v,l,t+1] <= omega[v,t+1]
                h.addRow(-highspy.kHighsInf, 0, 3,
                         np.array([binary_vars['w'][(v, l, t)], 
                                binary_vars['w'][(v, l, t_next)], 
                                binary_vars['omega'][(v, t_next)]], dtype=np.int32),
                         np.array([1.0, -1.0, -1.0], dtype=np.float64))
                constraint_count += 1
                nb20_count += 1

# NB21: No charging interruption (extended to full day)
for v in vehicles:
    for t in range(1, n_timesteps - 1):
        # chi[t-1] + chi[t+1] - 1 <= chi[t] + omega[t]
        h.addRow(-highspy.kHighsInf, 1, 4,
                 np.array([binary_vars['chi'][(v, t-1)], binary_vars['chi'][(v, t+1)],
                        binary_vars['chi'][(v, t)], binary_vars['omega'][(v, t)]], dtype=np.int32),
                 np.array([1.0, 1.0, -1.0, -1.0], dtype=np.float64))
        constraint_count += 1

# NB22-23: Linearization (delta_vr = x_vr * tau_v_ActrosL, phi_vrf = x_vr * tau_vf)
for v in vehicles:
    for r in range(n_routes):
        # Delta (diesel route)
        h.addRow(-highspy.kHighsInf, 0, 2,
                 np.array([binary_vars['delta'][(v, r)], binary_vars['x'][(v, r)]], dtype=np.int32),
                 np.array([1.0, -1.0], dtype=np.float64))
        h.addRow(-highspy.kHighsInf, 0, 2,
                 np.array([binary_vars['delta'][(v, r)], binary_vars['tau'][(v, 'ActrosL')]], dtype=np.int32),
                 np.array([1.0, -1.0], dtype=np.float64))
        h.addRow(-1, highspy.kHighsInf, 3,
                 np.array([binary_vars['delta'][(v, r)], binary_vars['x'][(v, r)], 
                        binary_vars['tau'][(v, 'ActrosL')]], dtype=np.int32),
                 np.array([1.0, -1.0, -1.0], dtype=np.float64))
        
        # Phi (type-route)
        for f in vehicle_types:
            h.addRow(-highspy.kHighsInf, 0, 2,
                     np.array([binary_vars['phi'][(v, r, f)], binary_vars['x'][(v, r)]], dtype=np.int32),
                     np.array([1.0, -1.0], dtype=np.float64))
            h.addRow(-highspy.kHighsInf, 0, 2,
                     np.array([binary_vars['phi'][(v, r, f)], binary_vars['tau'][(v, f)]], dtype=np.int32),
                     np.array([1.0, -1.0], dtype=np.float64))
            h.addRow(-1, highspy.kHighsInf, 3,
                     np.array([binary_vars['phi'][(v, r, f)], binary_vars['x'][(v, r)], 
                            binary_vars['tau'][(v, f)]], dtype=np.int32),
                     np.array([1.0, -1.0, -1.0], dtype=np.float64))
        constraint_count += 6

print(f"✓ NB19 (On-route): {n_vehicles * n_timesteps} constraints")
print(f"✓ NB20 (Night parking): {nb20_count} constraints")
print(f"✓ NB21 (No interruption): {n_vehicles * (n_timesteps - 2)} constraints")
print(f"✓ NB22-23 (Linearization): {6 * n_vehicles * n_routes} constraints")

### 5.24-5.26 Vollladen-Schutz und SOC-Prüfung - NEU!

In [ ]:
# NB24a: Full charge detection (strict binding without Big-M)
for v in vehicles:
    for t in timesteps:
        # mu * Q_max <= SOC
        q_max_indices = [binary_vars['tau'][(v, f)] for f in electric_types]
        q_max_values = [vehicle_params[vehicle_params['Type'] == f]['Q_max'].values[0] for f in electric_types]
        
        # This is complex in HiGHS - simplified version:
        # If mu=1, SOC >= Q_max (approximation with large coefficient)
        for f_idx, f in enumerate(electric_types):
            q_max_f = q_max_values[f_idx]
            # SOC >= mu * Q_max (when this type is assigned)
            h.addRow(0, highspy.kHighsInf, 3,
                     np.array([continuous_vars['SOC'][(v, t)], binary_vars['mu'][(v, t)], 
                            binary_vars['tau'][(v, f)]], dtype=np.int32),
                     np.array([1.0, -q_max_f, 0.0], dtype=np.float64))
        
        # mu <= epsilon (only for electric vehicles)
        h.addRow(-highspy.kHighsInf, 0, 2,
                 np.array([binary_vars['mu'][(v, t)], binary_vars['epsilon'][v]], dtype=np.int32),
                 np.array([1.0, -1.0], dtype=np.float64))
        constraint_count += len(electric_types) + 1

# NB24b: Charging interruption after full charge
for v in vehicles:
    for t in range(n_timesteps - 1):
        # chi[t+1] <= (1 - mu[t]) + omega[t+1]
        h.addRow(-highspy.kHighsInf, 1, 3,
                 np.array([binary_vars['chi'][(v, t+1)], binary_vars['mu'][(v, t)], 
                        binary_vars['omega'][(v, t+1)]], dtype=np.int32),
                 np.array([1.0, 1.0, -1.0], dtype=np.float64))
        constraint_count += 1

# NB25: SOC check at tour start (simplified)
for v in vehicles:
    for r in range(n_routes):
        t_start = route_params.iloc[r]['t_start']
        if 0 <= t_start < n_timesteps:
            # SOC at start >= energy needed
            d_r = route_params.iloc[r]['d_r']
            for f_idx, f in enumerate(electric_types):
                consumption = vehicle_params.loc[vehicle_params['Type'] == f, 'Consumption'].values[0]
                energy_needed = (consumption / 100) * d_r
                h.addRow(-highspy.kHighsInf, energy_needed, 2,
                         np.array([continuous_vars['SOC'][(v, t_start)], 
                                binary_vars['phi'][(v, r, f)]], dtype=np.int32),
                         np.array([1.0, -energy_needed], dtype=np.float64))
                constraint_count += 1

# NB26: Station switching prohibition
for v in vehicles:
    for t in range(n_timesteps - 1):
        # Sum(w[v,l,t] - w[v,l,t+1]) <= omega[v,t+1] + (1 - epsilon[v])
        indices_list = ([binary_vars['w'][(v, l, t)] for l in station_types] +
                  [binary_vars['w'][(v, l, t+1)] for l in station_types] +
                  [binary_vars['omega'][(v, t+1)], binary_vars['epsilon'][v]])
        values_list = ([1.0] * len(station_types) + [-1.0] * len(station_types) + [-1.0, 1.0])
        indices = np.array(indices_list, dtype=np.int32)
        values = np.array(values_list, dtype=np.float64)
        h.addRow(-highspy.kHighsInf, 1, len(values), indices, values)
        constraint_count += 1

print(f"✓ NB24a (Full charge detection): {n_vehicles * n_timesteps * (len(electric_types) + 1)} constraints")
print(f"✓ NB24b (No recharge after full): {n_vehicles * (n_timesteps - 1)} constraints")
print(f"✓ NB25 (SOC check at start): {n_vehicles * n_routes * len(electric_types)} constraints")
print(f"✓ NB26 (No station switching): {n_vehicles * (n_timesteps - 1)} constraints")

print("\n" + "=" * 60)
print(f"✓✓✓ TOTAL CONSTRAINTS ADDED: ~{constraint_count}")
print("=" * 60)

## 6. Solve the Model

HiGHS Solver mit 10 Minuten Zeitlimit und 1% MIP-Gap

In [ ]:
print("\n" + "=" * 70)
print("STARTING OPTIMIZATION")
print("=" * 70)
print(f"Variables: {h.getNumCol()}")
print(f"Constraints: {constraint_count}")
print(f"Time limit: 600s (10 minutes)")
print(f"MIP gap: 1%")
print("=" * 70)

# Run solver
status = h.run()

# Get results
model_status = h.getModelStatus()
info = h.getInfo()

print("\n" + "=" * 70)
print("OPTIMIZATION COMPLETE")
print("=" * 70)

if model_status == highspy.HighsModelStatus.kOptimal:
    print("✓ OPTIMAL SOLUTION FOUND!")
    print(f"  Objective value: {info.objective_function_value:,.2f} €/year")
    if info.mip_gap is not None:
        print(f"  MIP gap: {info.mip_gap * 100:.2f}%")
    solution_found = True
elif info.primal_solution_status:
    print("⚠ FEASIBLE SOLUTION FOUND (not proven optimal)")
    print(f"  Objective value: {info.objective_function_value:,.2f} €/year")
    if info.mip_gap is not None:
        print(f"  MIP gap: {info.mip_gap * 100:.2f}%")
    solution_found = True
else:
    print("✗ NO SOLUTION FOUND")
    print(f"  Status: {model_status}")
    solution_found = False

print("=" * 70)

## 7. Results Summary

Übersicht der optimalen Lösung

In [ ]:
if solution_found:
    # Get solution values
    solution = h.getSolution()
    
    # Helper function to get variable value
    def get_val(var_idx):
        return solution.col_value[var_idx]
    
    print("=" * 80)
    print("                        OPTIMALE LÖSUNG GEFUNDEN")
    print("=" * 80)
    print()
    
    # ========================================================================
    # 1. FLOTTENKOMPOSITION MIT DETAILS
    # ========================================================================
    print("╔" + "=" * 78 + "╗")
    print("║" + " " * 25 + "FLOTTENKOMPOSITION" + " " * 35 + "║")
    print("╠" + "=" * 78 + "╣")
    
    # Analyse der aktiven Fahrzeuge
    fleet_data = []
    for v in vehicles:
        if get_val(binary_vars['use'][v]) > 0.5:
            # Fahrzeugtyp ermitteln
            veh_type = None
            for f in vehicle_types:
                if get_val(binary_vars['tau'][(v, f)]) > 0.5:
                    veh_type = f
                    break
            
            # Zugewiesene Routen ermitteln
            routes_v = []
            for r in range(n_routes):
                if get_val(binary_vars['x'][(v, r)]) > 0.5:
                    routes_v.append(routes[r])
            
            # Gesamtkilometer berechnen
            total_km = sum(route_params.iloc[r]['d_r'] for r in range(n_routes) 
                          if get_val(binary_vars['x'][(v, r)]) > 0.5)
            
            fleet_data.append({
                'Fahrzeug': f"Fahrzeug {v+1:02d}",
                'Typ': veh_type,
                'Routen': len(routes_v),
                'km': total_km
            })
    
    # Zusammenfassung nach Typ
    n_diesel = sum(1 for d in fleet_data if d['Typ'] == 'ActrosL')
    n_e400 = sum(1 for d in fleet_data if d['Typ'] == 'eActros400')
    n_e600 = sum(1 for d in fleet_data if d['Typ'] == 'eActros600')
    
    print(f"║  Total aktive Fahrzeuge:     {len(fleet_data):>2}                                      ║")
    print(f"║  ├─ Diesel (ActrosL):        {n_diesel:>2}                                      ║")
    print(f"║  └─ Elektro:                 {n_e400 + n_e600:>2}                                      ║")
    if n_e400 > 0:
        print(f"║      ├─ eActros 400 (414 kWh): {n_e400:>2}                                      ║")
    if n_e600 > 0:
        print(f"║      └─ eActros 600 (621 kWh): {n_e600:>2}                                      ║")
    print("╠" + "=" * 78 + "╣")
    
    # Fahrzeugdetails als Tabelle
    if fleet_data:
        print("║ Fahrzeug      Typ             Anzahl Routen    Tages-km                      ║")
        print("║" + "-" * 78 + "║")
        for fd in fleet_data[:10]:  # Erste 10 Fahrzeuge
            print(f"║ {fd['Fahrzeug']:<12}  {fd['Typ']:<15} {fd['Routen']:>4}          {fd['km']:>6.0f} km               ║")
        if len(fleet_data) > 10:
            print(f"║ ... und {len(fleet_data)-10} weitere Fahrzeuge                                          ║")
    print("╚" + "=" * 78 + "╝")
    print()
    
    # ========================================================================
    # 2. LADEINFRASTRUKTUR
    # ========================================================================
    print("╔" + "=" * 78 + "╗")
    print("║" + " " * 25 + "LADEINFRASTRUKTUR" + " " * 36 + "║")
    print("╠" + "=" * 78 + "╣")
    
    installed_chargers = []
    for l_idx, l in enumerate(station_types):
        if get_val(binary_vars['y'][l]) > 0.5:
            spots = station_params.loc[l_idx, 'n_spots']
            power = station_params.loc[l_idx, 'P_max_l']
            capex = station_params.loc[l_idx, 'CAPEX_l']
            opex = station_params.loc[l_idx, 'OPEX_l']
            installed_chargers.append((l, spots, power, capex + opex))
    
    if installed_chargers:
        print("║ Ladesäulen-Typ         Ladepunkte  Max. Leistung    Kosten/Jahr           ║")
        print("║" + "-" * 78 + "║")
        for l, spots, power, cost in installed_chargers:
            print(f"║ {l:<22}     {spots}         {power:>4} kW        {cost:>8,.0f} €            ║")
        
        total_spots = sum(c[1] for c in installed_chargers)
        total_power = sum(c[2] for c in installed_chargers)
        total_cost = sum(c[3] for c in installed_chargers)
        
        print("║" + "-" * 78 + "║")
        print(f"║ GESAMT:                   {total_spots}         {total_power:>4} kW        {total_cost:>8,.0f} €            ║")
    else:
        print("║ Keine Ladeinfrastruktur (reine Diesel-Flotte)                             ║")
    
    print("╚" + "=" * 78 + "╝")
    print()
    
    # ========================================================================
    # 3. NETZANSCHLUSS & SPEICHER
    # ========================================================================
    print("╔" + "=" * 78 + "╗")
    print("║" + " " * 23 + "NETZANSCHLUSS & SPEICHER" + " " * 31 + "║")
    print("╠" + "=" * 78 + "╣")
    
    grid_extended = get_val(binary_vars['gamma']) > 0.5
    peak_power = get_val(continuous_vars['P_peak'])
    storage_p = get_val(continuous_vars['P_sp'])
    storage_e = get_val(continuous_vars['E_sp'])
    
    grid_capacity = P_netz_basis + (P_netz_erw if grid_extended else 0)
    grid_status = "(erweitert +500 kW)" if grid_extended else "(Basis)"
    
    print(f"║ Netzanschluss:            {grid_capacity:>6.0f} kW  {grid_status:<20}        ║")
    print(f"║ Spitzenlast:              {peak_power:>6.0f} kW                                    ║")
    print("║" + "-" * 78 + "║")
    
    if storage_p > 0.1 or storage_e > 0.1:
        print(f"║ Batteriespeicher:         {storage_p:>6.0f} kW  /  {storage_e:>6.0f} kWh                   ║")
        storage_cost = (1 + alpha_opex) * (c_sp_p * storage_p + c_sp_e * storage_e)
        print(f"║ Speicher-Kosten:          {storage_cost:>8,.0f} €/Jahr                           ║")
    else:
        print("║ Batteriespeicher:         NICHT INSTALLIERT                                 ║")
    
    print("╚" + "=" * 78 + "╝")
    print()
    
    # ========================================================================
    # 4. GESAMTKOSTEN
    # ========================================================================
    total_cost = info.objective_function_value
    
    print("╔" + "=" * 78 + "╗")
    print("║" + " " * 26 + "GESAMTKOSTEN (JÄHRLICH)" + " " * 30 + "║")
    print("╠" + "=" * 78 + "╣")
    print(f"║                                                                              ║")
    print(f"║                {total_cost:>15,.2f} EUR / Jahr                           ║")
    print(f"║                                                                              ║")
    print("╚" + "=" * 78 + "╝")
    print()
    
    # Optimierungsstatus
    if info.mip_gap is not None:
        print(f"MIP Gap: {info.mip_gap * 100:.2f}%")
    
else:
    print("\n" + "=" * 80)
    print("KEINE OPTIMALE LÖSUNG GEFUNDEN!")
    print("=" * 80)

## 8. Visualizations

Detaillierte Visualisierungen der Lösung

In [ ]:
if solution_found:
    # ========================================================================
    # 8.1 SOC-VERLÄUFE DER ELEKTRO-LKW
    # ========================================================================
    
    # Identifiziere Elektro-Fahrzeuge
    electric_vehicles = []
    for v in vehicles:
        if get_val(binary_vars['use'][v]) > 0.5:
            if get_val(binary_vars['epsilon'][v]) > 0.5:
                # Finde den Typ
                veh_type = None
                for f in electric_types:
                    if get_val(binary_vars['tau'][(v, f)]) > 0.5:
                        veh_type = f
                        break
                if veh_type:
                    electric_vehicles.append((v, veh_type))
    
    if electric_vehicles:
        n_elec = len(electric_vehicles)
        n_cols = min(3, n_elec)
        n_rows = (n_elec + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows), squeeze=False)
        colors = plt.cm.tab10(np.linspace(0, 1, n_elec))
        
        for idx, (v, veh_type) in enumerate(electric_vehicles):
            row = idx // n_cols
            col = idx % n_cols
            ax = axes[row, col]
            
            # SOC-Werte extrahieren
            hours = [t * 0.25 for t in range(n_timesteps)]
            soc_values = [get_val(continuous_vars['SOC'][(v, t)]) for t in range(n_timesteps)]
            
            # Als Prozent
            q_max = vehicle_params[vehicle_params['Type'] == veh_type]['Q_max'].values[0]
            soc_percent = [s / q_max * 100 for s in soc_values]
            
            # Plot
            ax.plot(hours, soc_percent, color=colors[idx], linewidth=2)
            ax.fill_between(hours, soc_percent, alpha=0.3, color=colors[idx])
            ax.axhline(y=10, color='red', linestyle='--', linewidth=1, label='10% Min')
            
            ax.set_xlim(0, 24)
            ax.set_ylim(0, 105)
            ax.set_xlabel('Uhrzeit [h]')
            ax.set_ylabel('SOC [%]')
            ax.set_title(f'Fahrzeug {v+1} ({veh_type})', fontweight='bold')
            ax.grid(True, alpha=0.3)
            
            # Statistik anzeigen
            min_soc = min(soc_percent)
            max_soc = max(soc_percent)
            ax.text(0.02, 0.02, f'Min: {min_soc:.1f}%\nMax: {max_soc:.1f}%', 
                   transform=ax.transAxes, fontsize=8, verticalalignment='bottom',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Leere Subplots ausblenden
        for idx in range(n_elec, n_rows * n_cols):
            axes[idx // n_cols, idx % n_cols].set_visible(False)
        
        plt.suptitle('SOC-VERLÄUFE DER ELEKTRO-LKW', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Tabelle mit SOC-Statistik
        print("\n" + "=" * 70)
        print("SOC-STATISTIK PRO E-LKW")
        print("=" * 70)
        print(f"{'Fahrzeug':<12} {'Typ':<12} {'Batterie':>10} {'Min SOC':>10} {'Max SOC':>10}")
        print("-" * 70)
        for v, veh_type in electric_vehicles:
            soc_values = [get_val(continuous_vars['SOC'][(v, t)]) for t in range(n_timesteps)]
            q_max = vehicle_params[vehicle_params['Type'] == veh_type]['Q_max'].values[0]
            min_pct = min(soc_values) / q_max * 100
            max_pct = max(soc_values) / q_max * 100
            print(f"Fahrzeug {v+1:02d}  {veh_type:<12} {q_max:>7.0f} kWh {min_pct:>8.1f}% {max_pct:>8.1f}%")
    else:
        print("\nKeine E-LKW in der Flotte.")
    
    # ========================================================================
    # 8.2 NETZLAST ÜBER 24 STUNDEN
    # ========================================================================
    
    print("\n" + "=" * 70)
    print("NETZLAST-ANALYSE")
    print("=" * 70)
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Netzlast extrahieren
    hours = [t * 0.25 for t in range(n_timesteps)]
    grid_power = [get_val(continuous_vars['p_netz'][t]) for t in range(n_timesteps)]
    peak_val = get_val(continuous_vars['P_peak'])
    
    # Plot
    ax.fill_between(hours, grid_power, alpha=0.5, color='blue', label='Netzlast')
    ax.plot(hours, grid_power, 'b-', linewidth=2)
    ax.axhline(y=peak_val, color='red', linestyle='--', linewidth=2, 
               label=f'Spitzenlast ({peak_val:.0f} kW)')
    
    # Netzlimit
    grid_extended = get_val(binary_vars['gamma']) > 0.5
    grid_limit = P_netz_basis + (P_netz_erw if grid_extended else 0)
    ax.axhline(y=grid_limit, color='green', linestyle=':', linewidth=2, 
               label=f'Netzlimit ({grid_limit} kW)')
    
    ax.set_xlim(0, 24)
    ax.set_xlabel('Uhrzeit [h]')
    ax.set_ylabel('Leistung [kW]')
    ax.set_title('NETZLAST ÜBER 24 STUNDEN', fontsize=14, fontweight='bold')
    ax.set_xticks(range(0, 25, 2))
    ax.set_xticklabels([f'{h:02d}:00' for h in range(0, 25, 2)])
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    # Nachtzeit markieren (18:00-06:00)
    ax.axvspan(0, 6, alpha=0.1, color='gray', label='Nachtzeit')
    ax.axvspan(18, 24, alpha=0.1, color='gray')
    
    plt.tight_layout()
    plt.show()
    
    # Stromkosten-Zusammenfassung
    daily_energy = sum(get_val(continuous_vars['p_netz'][t]) * delta_t for t in range(n_timesteps))
    yearly_energy = daily_energy * D
    energy_cost = yearly_energy * p_arbeit
    power_cost = peak_val * p_leistung
    
    print(f"\nTäglicher Energiebedarf:  {daily_energy:>10.1f} kWh")
    print(f"Jährlicher Energiebedarf: {yearly_energy:>10,.1f} kWh")
    print(f"Spitzenlast:              {peak_val:>10.1f} kW")
    print("-" * 70)
    print(f"Arbeitspreis ({p_arbeit} €/kWh): {energy_cost:>12,.2f} €")
    print(f"Leistungspreis ({p_leistung} €/kW): {power_cost:>12,.2f} €")
    print(f"Grundgebühr:              {p_grund:>12,.2f} €")
    print("-" * 70)
    print(f"STROMKOSTEN GESAMT:       {energy_cost + power_cost + p_grund:>12,.2f} €/Jahr")
    
else:
    print("Keine Lösung verfügbar für Visualisierung.")